In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

import numpy as np

import PIL
from PIL import Image

import time

import os

# Extra imports
import io
from tensorflow.python.data.experimental import AUTOTUNE

In [ ]:
!gdown https://drive.google.com/u/0/uc?id=1STYsoP85lyKAtarMRuDyTjp89tAbIDM-

Downloading...
From: https://drive.google.com/u/0/uc?id=1STYsoP85lyKAtarMRuDyTjp89tAbIDM-
To: /content/caltech256_subset_resized_cropped256x256.zip
100% 287M/287M [00:03<00:00, 72.9MB/s]


In [ ]:
!unzip -o caltech256_subset_resized_cropped256x256.zip

Streaming output truncated to the last 5000 lines.
  inflating: caltech256_subset_resized_cropped256x256/data/205.superman/205_0014.jpg  
  inflating: caltech256_subset_resized_cropped256x256/data/205.superman/205_0015.jpg  
  inflating: caltech256_subset_resized_cropped256x256/data/205.superman/205_0016.jpg  
  inflating: caltech256_subset_resized_cropped256x256/data/205.superman/205_0017.jpg  
  inflating: caltech256_subset_resized_cropped256x256/data/205.superman/205_0018.jpg  
  inflating: caltech256_subset_resized_cropped256x256/data/205.superman/205_0019.jpg  
  inflating: caltech256_subset_resized_cropped256x256/data/205.superman/205_0020.jpg  
  inflating: caltech256_subset_resized_cropped256x256/data/205.superman/205_0021.jpg  
  inflating: caltech256_subset_resized_cropped256x256/data/205.superman/205_0022.jpg  
  inflating: caltech256_subset_resized_cropped256x256/data/205.superman/205_0023.jpg  
  inflating: caltech256_subset_resized_cropped256x256/data/205.superman/205_002

In [ ]:
shuffled_paths = [] 
shuffled_labels = []

with open("caltech256_subset_resized_cropped256x256/shuffled_labels.txt") as label_file:
    label_file_lines = label_file.readlines()
    
for line in label_file_lines:
    image_path, image_label = line.strip().split(",")
    shuffled_paths.append(image_path)
    shuffled_labels.append(int(image_label))

In [ ]:
with tf.io.TFRecordWriter("caltech_dataset.tfrecords") as writer:
    for path, label in zip(shuffled_paths, shuffled_labels):
        image = Image.open("caltech256_subset_resized_cropped256x256/data/" + path)

        bytes_buffer = io.BytesIO()
        image.convert("RGB").save(bytes_buffer, "JPEG")
        image_bytes = bytes_buffer.getvalue()

        bytes_feature = tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))
        class_feature = tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))

        example = tf.train.Example(
          features=tf.train.Features(feature={
              "image": bytes_feature,
              "class": class_feature
          })
        )

        writer.write(example.SerializeToString())

        image.close()

In [ ]:
image_feature_description = {
    "image": tf.io.FixedLenFeature([], tf.string), 
    "class": tf.io.FixedLenFeature([], tf.int64), 
    }

In [ ]:
def _parse_data(unparsed_example):
    return tf.io.parse_single_example(unparsed_example, image_feature_description)

In [ ]:
def _bytestring_to_pixels(parsed_example):
    byte_string = parsed_example['image']
    image = tf.io.decode_image(byte_string)
    image = tf.reshape(image, [256, 256, 3])
    return image, parsed_example["class"]

In [ ]:
def load_and_extract_images(filepath):
    dataset = tf.data.TFRecordDataset(filepath)
    dataset = dataset.map(_parse_data, num_parallel_calls=AUTOTUNE)
    dataset = dataset.map(_bytestring_to_pixels, num_parallel_calls=AUTOTUNE) # .cache()
    return dataset

In [ ]:
caltech_dataset = load_and_extract_images("caltech_dataset.tfrecords")

In [ ]:
train_split = 0.6
validation_split = 0.2

num_train_images = int(len(shuffled_paths) * train_split)
num_validation_images = int(len(shuffled_paths) * validation_split)

In [ ]:
train_dataset = caltech_dataset.take(num_train_images)

In [ ]:
validation_dataset = caltech_dataset.skip(num_train_images).take(num_validation_images)

In [ ]:
test_dataset = caltech_dataset.skip(num_train_images + num_validation_images)